<a href="https://colab.research.google.com/github/lilyhuong/Hadoop_spark/blob/master/Traitement_des_validations_du_re%CC%81seau_de_transport_dIle_de_France_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-projet Apache Spark

Merci d'indiquer les noms composant le binôme :

|Nom | Prénom|
|---|---|
| Nguyen | Thi Huong |
| Stary | Joffrey |

L'objectif est de collecter les données de validations quotidiennes de titres de transport de la région parisienne, disponibles en Open Data sur le site de Mobilités Ile-de-France.

On se concentrera sur le **réseau ferré** exclusivement pour cet exercice.

## Installation de Spark ⚡

Apache Spark n'est pas disponible en standard sur Google Colab.
Procéder à son installation, ainsi qu'à son initialisation pour réaliser le traitement à venir.

In [2]:
# Installation de pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 49 kB/s 
     |████████████████████████████████| 199 kB 63.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=36f53f14c8fe507866ebeb3a30e8713aada2203cd5bfc02912f8b041ae3f5d05
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
# Importation de pyspark
import pyspark

In [4]:
# Création de la session
spark = pyspark.sql.SparkSession.builder.appName('Project 2022').getOrCreate()

In [6]:
spark

## Récupération des données 🚆

Sur le site https://data.iledefrance-mobilites.fr, récupérer les données de validation par jour.

On récupérera les fichiers des validations sur le réseau de surface disponibles ici [https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/table/](https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/table/). On prendra les années 2020 et 2021 pour l'exercice :
* data-rf-2020.zip
* data-rf-2021.zip

Identifiez les données de S1 2020 à S2 2021 pour disposer d'un historique de deux ans.

On utilisera pour ce faire les commandes de téléchargement de fichiers (`curl`) depuis un site (pas de chargement manuel).

__Attention__ : prévoir quelques minutes pour le téléchargement, au moins une première fois, et donc une copie sur Google Drive si Google Colab est utilisée, afin d'éviter ce temps d'attente lors de sessions de travail successives.

Les fichiers sont des zip. Ils peuvent être décompressés avec la commande `unzip`.

Ces fichiers seront localisés dans un sous-répertoire.

__Documentation de curl__ : http://pwet.fr/man/linux/commandes/curl/

## **Notre réponse** 

Afin de télécharger les données, nous utilisons les liens disponibles sur le site d'Île-de-France mobilités. Les fichiers obtenus sont au format .zip ce qui nécessite de les unzip. Ce que nous faisons en reprenant les fonctions que vous nous conseillez. On obtient finalement les deux folders : ***data-rf-2020*** & ***data-rf-2021*** contenant les informations relatives à la fréquentation mesurée par semestre.

L'étude d'un document explicatif ([Présentation des données Open Data](https://data.iledefrance-mobilites.fr/api/datasets/1.0/histo-validations-reseau-ferre/attachments/donnees_de_validation_pdf/)) nous amène à considérer ***NB_FER*** comme le jeu de données qui nous sera utile pour la suite du projet. Ce dernier nous renseigne sur le nombre de validation par jour, par catégorie de titre et par station sur le réseau férré îlien.

Afin que les données restent disponibles au téléchargement, nous avons créé un dépôt sur ***github*** depuis lequel les données sont accessibles. Si la première partie du code ne fonctionne pas en raison d'une impossibilité à télécharger depuis Île-de-France mobilités, alors utilisez la partie de code "***Alternative***".

In [7]:
# Téléchargement répertoire 2020
!curl -o file_2020.zip https://data.iledefrance-mobilites.fr/api/v2/catalog/datasets/histo-validations-reseau-ferre/files/e6bcf4c994951fc086e31db6819a3448
# Téléchargement répertoire 2021
!curl -o file_2021.zip https://data.iledefrance-mobilites.fr/api/v2/catalog/datasets/histo-validations-reseau-ferre/files/e35b9ec0a183a8f2c7a8537dd43b124c

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 82.8M    0 82.8M    0     0  31.9M      0 --:--:--  0:00:02 --:--:-- 31.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.0M    0 16.0M    0     0  29.4M      0 --:--:-- --:--:-- --:--:-- 29.4M


In [8]:
# On décompresse les fichiers qui sont au format .zip
  # Si le fichier existe déjà alors on le remplace
!unzip -o /content/file_2020.zip
!unzip -o /content/file_2021.zip

Archive:  /content/file_2020.zip
  inflating: data-rf-2020/2020_S1_NB_FER.txt  
  inflating: data-rf-2020/2020_S1_PROFIL_FER.txt  
  inflating: data-rf-2020/2020_S2_NB_FER.txt  
  inflating: data-rf-2020/2020_S2_PROFIL_FER.txt  
 extracting: data-rf-2015.zip        
 extracting: data-rf-2016.zip        
 extracting: data-rf-2017.zip        
 extracting: data-rf-2018.zip        
 extracting: data-rf-2019.zip        
Archive:  /content/file_2021.zip
  inflating: data-rf-2021/2021_S1_NB_FER.txt  
  inflating: data-rf-2021/2021_S1_PROFIL_FER.txt  
  inflating: data-rf-2021/2021_S2_NB_FER.txt  
  inflating: data-rf-2021/2021_S2_PROFIL_FER.txt  


In [10]:
import os

In [12]:
os.getcwd()

'/content'

In [9]:
#### Améliorer avec une liste et vérifier que ça existe!!!!

# Importation os
import os

# Mise au format .csv
os.rename('/content/data-rf-2020/2020_S1_NB_FER.txt', '/content/data-rf-2020/2020_S1_NB_FER.csv')
os.rename('/content/data-rf-2020/2020_S2_NB_FER.txt', '/content/data-rf-2020/2020_S2_NB_FER.csv')
os.rename('/content/data-rf-2021/2021_S1_NB_FER.txt', '/content/data-rf-2021/2021_S1_NB_FER.csv')
os.rename('/content/data-rf-2021/2021_S2_NB_FER.txt', '/content/data-rf-2021/2021_S2_NB_FER.csv')

## **Alternative en cas d'inaccessibilité des données** ⛔

Le code qui suit permet de télécharger les jeux de données si ils n'étaient pas disponible sur Île-de-France mobilité.

**Ne faire tourner que si nécessaire. ❗**

In [18]:
# Téléchargement des jeux de données depuis un github hébergeant les jeux de données pour les 4 semestres au format .csv
  # Ces fichiers ont été obtenus initiallement depuis la partie précédent du code et une extraction manuelle depuis google collab vers le github

!curl -o S1_2020 https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2020_S1_NB_FER.csv
!curl -o S2_2020 https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2020_S2_NB_FER.csv
!curl -o S1_2021 https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2021_S1_NB_FER.csv
!curl -o S2_2021 https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2021_S2_NB_FER.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47.8M  100 47.8M    0     0  20.5M      0  0:00:02  0:00:02 --:--:-- 20.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 56.8M  100 56.8M    0     0  20.9M      0  0:00:02  0:00:02 --:--:-- 20.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.5M  100 57.5M    0     0  19.2M      0  0:00:02  0:00:02 --:--:-- 19.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.7M  100 58.7M    0     0  18.9M      0  0:00:03  0:00:03 --:--:-- 18.9M


In [ ]:
!curl -O https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2020_S1_NB_FER.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47.8M  100 47.8M    0     0  3725k      0  0:00:13  0:00:13 --:--:-- 12.1M


In [ ]:
!curl -O https://raw.githubusercontent.com/lilyhuong/Hadoop_spark/master/2021_S2_NB_FER.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.7M  100 58.7M    0     0  20.0M      0  0:00:02  0:00:02 --:--:-- 20.0M


In [ ]:
!head 2021_S2_NB_FER.csv

JOUR	CODE_STIF_TRNS	CODE_STIF_RES	CODE_STIF_ARRET	LIBELLE_ARRET	ID_REFA_LDA	CATEGORIE_TITRE	NB_VALD
01/07/2021	100	110	1	PORTE MAILLOT	71379	?	204
01/07/2021	100	110	1	PORTE MAILLOT	71379	AMETHYSTE	157
01/07/2021	100	110	1	PORTE MAILLOT	71379	AUTRE TITRE	983
01/07/2021	100	110	1	PORTE MAILLOT	71379	FGT	304
01/07/2021	100	110	1	PORTE MAILLOT	71379	IMAGINE R	1958
01/07/2021	100	110	1	PORTE MAILLOT	71379	NAVIGO	8821
01/07/2021	100	110	1	PORTE MAILLOT	71379	NAVIGO JOUR	43
01/07/2021	100	110	1	PORTE MAILLOT	71379	NON DEFINI	766
01/07/2021	100	110	1	PORTE MAILLOT	71379	TST	613


### Commentaires sur les données disponibles sur ce portail

Investiguer les données diponibles sur le portail.

Question : peut-on constituer un historique de données s'étendant sur les trois dernières années (2019 à 2021) ?


## **Notre réponse**

Le portail Île-de-France mobilités offre des données sur la fréquentation mesurée du réseau ferré pour la période 2015-2021 (voir : [Historique des données de validation sur le réseau ferré (2015-2021)](https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B)). La structure des fichiers est la même pour toutes ces années ce qui rend l'ajout de l'année 2019 dans l'historique simple, il suffit d'ajouter le téléchargement et l'extraction du jeu de données pour 2019 pour établir cet historique. La méthode reste similaire à ce qui a été fait précédemment pour 2020 et 2021. Il en est de même pour toute la période couverte par ce jeu de données (soit 2015-2021 en Novembre 2022).

***La création d'une historique pour la période 2019-2021 est donc possible et implémentable sans difficultés.***

On peut d'ailleurs ajouter que les données sont mises à jour à un rythme bi-annuel (dernière mise à jour en octobre 2022 - [source](https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/information/)). Il serait donc possible de continuellement suivre cet historique avec la mise à jour des données en OPEN DATA, à condition que la politique de diffusion des données et leur nature ne soit pas altérés.

Ci-dessous le lien pour télécharger les données de 2019 :

> [Données pour 2019](https://data.iledefrance-mobilites.fr/api/v2/catalog/datasets/histo-validations-reseau-ferre/files/6d7e7b859e6acac7bebad18bdb37bfc3)



## Lecture des fichiers dans Spark 📁

Lire les fichiers en choisissant les bonnes options de lecture.
Concaténer les données en une seule table.

## **Notre réponse**

Les fichiers ont été préalablement passés au format .csv où la ***tabulation*** ('\t') sert de séparateur. Le nom des variables est quant à lui présent en header.

Nous créons ***un dataframe par semestre***. En affichant les 5 premières lignes de ces dataframes, on voit que la structure, les variables présentes sont similaires.

On ***concatène*** ensuite les dataframes ***par année*** pour obtenir un dataframe par année avant de le faire pour la période avec ***df_validations***, le dataframe ***pour toute la période*** qui sera utilisé pour la suite de l'analyse.

In [17]:
# Création d'un dataframe par semestre entre S1 2020 et S2 2021
df_2020_S1 = spark.read.csv('/content/data-rf-2020/2020_S1_NB_FER.csv', sep = '\t', header = True)
df_2020_S2 = spark.read.csv('/content/data-rf-2020/2020_S2_NB_FER.csv', sep = '\t', header = True)
df_2021_S1 = spark.read.csv('/content/data-rf-2021/2021_S1_NB_FER.csv', sep = '\t', header = True)
df_2021_S2 = spark.read.csv('/content/data-rf-2021/2021_S2_NB_FER.csv', sep = '\t', header = True)

In [16]:
df_2020_S1 = spark.read.text('/content/data-rf-2020/2020_S1_NB_FER.txt')

AnalysisException: ignored

In [ ]:
# Afficher les têtes des data frames (par semestre)
df_2020_S1.show(5)
df_2020_S2.show(5)
df_2021_S1.show(5)
df_2021_S2.show(5)

+----------+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|      JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----------+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|              ?|     28|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|      AMETHYSTE|     83|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|    AUTRE TITRE|     94|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|            FGT|     84|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|      IMAGINE R|    522|
+----------+--------------+-------------+---------------+-------------+-----------+---------------+-------+
only showing top 5 rows

+--

In [ ]:
# Concaténations des données par année
df_2020 = df_2020_S1.unionByName(df_2020_S2)
df_2021 = df_2021_S1.unionByName(df_2021_S2)

# Concaténations des données sur la période
df_validations = df_2020.unionByName(df_2021)

In [ ]:
# Afficher les têtes des data frames (par année)
df_2020.show(10)
df_2021.show(10)

# Afficher les têtes des data frames (sur la période)
df_validations.show(20)

+----------+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|      JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----------+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|              ?|     28|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|      AMETHYSTE|     83|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|    AUTRE TITRE|     94|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|            FGT|     84|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|      IMAGINE R|    522|
|01/01/2020|           100|          110|              1|PORTE MAILLOT|      71379|         NAVIGO|   2049|
|01/01/2020|           100| 

## Préparation des données

Réaliser les transformations nécessaires pour exploiter ces données :
- préparation des dates (champ JOUR)

Quelle est la tranformation à réaliser ?

## **Notre réponse**

TO WRITE Expliquer la démarche

In [ ]:
  # Probablement inutile de traiter autre chose que JOUR
    # On ne se sert pas trop des autres vars
    # Le point ? c'est normal
  
  # De Joffrey
    # Je pense qu'il faut UNIQUEMENT traiter JOUR
      # Bon format,
      # Vérifier si miss
      # Vérifier si outliers (des jours hors de la période 1er janvier 2020 à 31 décembre 2021)

  # Mais par la suite, besoin de quelques traitements...

On va regarder la structure de dataframe 

In [ ]:
# Affichage de la structure du dataframe
df_validations.printSchema()

root
 |-- JOUR: string (nullable = true)
 |-- CODE_STIF_TRNS: string (nullable = true)
 |-- CODE_STIF_RES: string (nullable = true)
 |-- CODE_STIF_ARRET: string (nullable = true)
 |-- LIBELLE_ARRET: string (nullable = true)
 |-- ID_REFA_LDA: string (nullable = true)
 |-- CATEGORIE_TITRE: string (nullable = true)
 |-- NB_VALD: string (nullable = true)



* Pour la variable jour, on doit convert en type datetime, NB_VALD to numeric 
* Check les valeurs manquants pour chaque variable => chercher une solution
* Faire une descriptive statistic pour vérifier si il y a des outliers dans notre base de donnée (notatement dans la colonne NB VALID


## **Préparation des données**

En plus da la variable ***JOUR***, nous préparons dès à présent certaines variables.



*   ***Valeurs manquantes***
  *   On regarde pour toutes les varibles si il y a des valeurs manquantes.

*   ***NB_VALD***
  *   Cette variable donne le nombre de validations ce qui implique d'en faire une variable numérique (entière car il ne peut pas y avoir de parti de validations, un passage est validé ou non).
  *   Solution - Passage en ***integer***.





In [ ]:
# Ne marche pas ça reste string, revoir
import pyspark.sql.functions as F

In [ ]:
# On regarde si il y a des valeurs manquantes
from pyspark.sql.functions import col,isnan, when, count

df_validations.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_validations.columns]
   ).show()

+----+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----+--------------+-------------+---------------+-------------+-----------+---------------+-------+
|   0|             0|            0|              0|            0|          0|              0|      0|
+----+--------------+-------------+---------------+-------------+-----------+---------------+-------+



In [ ]:
# Passer NB_VALD en integer
#from pyspark.sql.functions import withColumn

df_validations = df_validations.withColumn("NB_VALD", df_validations.NB_VALD.cast('int'))

ImportError: ignored

In [ ]:
df.printSchema()

root
 |-- JOUR: string (nullable = true)
 |-- CODE_STIF_TRNS: string (nullable = true)
 |-- CODE_STIF_RES: string (nullable = true)
 |-- CODE_STIF_ARRET: string (nullable = true)
 |-- LIBELLE_ARRET: string (nullable = true)
 |-- ID_REFA_LDA: string (nullable = true)
 |-- CATEGORIE_TITRE: string (nullable = true)
 |-- NB_VALD: string (nullable = true)



## Détermination des principales catégories de titre

Différentes catégories de titre sont utilisées sur le réseau.

Déterminer les deux catégories principalement utilisées. Seules ces catégories seront utilisées dans les travaux ci-après (les utiliser comme filtre sur les validations dans la suite).

## **Notre réponse**

ffff

In [ ]:
# Votre code ici

In [ ]:
# To Do List - Joffrey
  # CATEGORIE_TITRE
    # Liste des catégories
    # Trouver les 2 catégories qui reviennent le plus
  # Réutiliser ces 2 catégories plus la SUITE
    # Créer un df avec uniquement les 2 catégories les plus utilisées

In [ ]:
# Importation des fonctions SQL pyspark
import pyspark.sql.functions

df_validations.groupby('CATEGORIE_TITRE')



## Visualisation du trafic dans une station

Visualiser le trafic à la gare de Lyon pour les deux catégories de titre principales.

Attention à gérer le cas des gares (comme la gare de Lyon) présentes sur plusieurs lignes et dont le libellé apparaît donc sur plusieurs lignes. Investiguer ce cas avant de déterminer la bonne façon de calculer le nombre de validations pour la gare de Lyon.

In [ ]:
# Votre code mettant en évidence le cas des gares sur plusieurs lignes

In [ ]:
# LiLy
  
  # Un même lieu (gare) est présent sur plusieurs lignes (plusieurs libellés)
  # Attention à bien le faire par année et semestre
  # https://data.iledefrance-mobilites.fr/api/datasets/1.0/histo-validations-reseau-ferre/attachments/donnees_de_validation_pdf/

# Etape 1
  # Montrer pour un libellé (gare de lyon) qu'il existe différents codes d'arrêts

# Etape 2
  # On veut additionner les validation pour :
    # une même catégorie
    # CODE_STIF_ARRET différent mais même LIBELLE_ARRET & même jour
    # ==> Groupby NBvalidation par catégorie/jour/libelle d'arrêt

In [ ]:
# Votre code visualisant le trafic à la gare de Lyon

In [ ]:
# LiLy
  
  # Graphique (pandas, matplotlib), difficile sur spark (impossible ?)

## Fluctuation du trafic hebdomadaire

Calculer le trafic total et le pourcentage par jour de la semaine sur l'ensemble du réseau.

Trier le résultat par ordre décroissant de validations.

Note : considérer l'usage d'une fonction analytique (`Window.partitionBy()`).


In [ ]:
# Votre code ici

In [ ]:
# Joffrey

# Etape 1
  # Créer une variable (1 à 7) renvoyant à un jour de la semaine (fonction dayofweek)
  # L'attribuer à toutes nos observations

# Etape 2
  # SUM les validations (2 catégories réunies) pour une journée, la somme des validations pour le lundi par exemple
  # % des validations d'une journée par rapport à la semaine (Lundi/semaine)
  # Trier par ordre décroissant
  # graphique ?
  
  # SUM des validations par jour (Lundi à Dimanche)
  # % des validations d'une journée ramenée à la semaine

### Impact du télétravail

Mettre en évidence l'impact du télétravail en comparant les comportements hebdomadaires de janvier et février 2020 et ceux de janvier et février 2021.

In [ ]:
# Votre code ici

In [ ]:
  # Descriptive puis graphique
  # 2 catégories réunies ??

  # Etape 1
    # Comparer les 2 périodes

## Analyse de l'impact du reconfinement d'octobre 2020

Mettre en évidence graphiquement l'impact du reconfinement.

N'utiliser que les catégories de titre _IMAGINE R_ et _Navigo_.


In [ ]:
# Votre code ici

In [ ]:
  # Graphique avec une barre au moment du reconfinement + mettre 2021 pour comparer ?
  # Plusieurs graphiques
    # fusionnés
    # courbes
    # barplot
    # cumulé (surface rouge cat 1 et au dessus en bleu la cat 2)

#### Bonus

Calculer la moyenne glissante sur 7 jours par categorie de titre pour réduire les variations hebdomadaires.

In [ ]:
# Votre code ici

In [ ]:
  # Et représenter graphiquement
  # Sur TOUTE la période

## Modélisation avec Apache Spark

On essaie de faire un modèle basique de prévision du trafic dans les 7 prochains jours, pour une station.

Apache Spark MLlib n'intègre pas de modèle pour les séries chronologiques.

L'approche classique est alors d'utiliser une technique de régression classique (régression linéaire bien sûr, mais aussi RandomForestRegressor par exemple).

Pour une première version simple, utiliser un vecteur constituer des validations sur les 14 jours précédents (X) pour prédire les validations du jour (y). Dans cette version, on utilisera une `LinearRegression` ou un `RandomForestRegressor`, au choix.

Le code doit comporter :
- la préparation des _features_ (X)
- la constitution d'un ensemble d'apprentissage et de test
- l'entrainement d'un modèle
- le mesure de la performance du modèle : RMSE

Rappel : ne travailler que sur les deux catégories de titre principales.


In [ ]:
# Votre code ici

## Amélioration du modèle

Discuter des façons d'améliorer cette première version du modèle.

On pourra se reporter par exemple à l'article suivant pour l'usage des forêts aléatoires sur ce type de problème : https://arxiv.org/abs/2101.02118

**Remplir votre commentaire ici**

## OPTION : implémentation des améliorations

Implémenter tout ou partir des suggestions d'amélioration.

**Cette partie est facultative et sera bonifiante si les éléments contribués sont probants.**

In [ ]:
# votre code ici